<a href="https://colab.research.google.com/github/khawajaMuzammil/CodeAlpha_Project_Basic-Chatbot/blob/main/DNA_MATCHING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install -y openmpi-bin libopenmpi-dev

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
%%writefile dna_search.cu
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <cuda_runtime.h>

// ========== CUDA KERNELS ==========

// Kernel 1: Generate DNA sequences
__global__ void generate_dna_kernel(char* sequences, int total_seqs, int seq_len,
                                   char* pattern, int pat_len, int seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < total_seqs) {
        // Each thread generates one DNA sequence
        unsigned int local_seed = seed + idx * 12345;

        // Fill with random ATCG
        for (int i = 0; i < seq_len; i++) {
            int r = (local_seed = local_seed * 1103515245 + 12345) % 4;
            sequences[idx * seq_len + i] = "ATCG"[r];
        }

        // Insert pattern in first 100 sequences (guaranteed matches)
        if (idx < 100 && pat_len > 0) {
            int pos = (local_seed = local_seed * 1103515245 + 12345) % (seq_len - pat_len);
            for (int i = 0; i < pat_len; i++) {
                sequences[idx * seq_len + pos + i] = pattern[i];
            }
        }
    }
}

// Kernel 2: Search pattern in DNA
__global__ void search_kernel(char* sequences, char* pattern, int* results,
                             int seq_len, int pat_len, int num_seqs) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < num_seqs) {
        int count = 0;
        char* seq = &sequences[idx * seq_len];

        // Slide pattern through sequence
        for (int i = 0; i <= seq_len - pat_len; i++) {
            bool match = true;
            for (int j = 0; j < pat_len; j++) {
                if (seq[i + j] != pattern[j]) {
                    match = false;
                    break;
                }
            }
            if (match) count++;
        }
        results[idx] = count;
    }
}

// ========== HELPER FUNCTIONS ==========

void save_dna_to_file(char* sequences, int total_seqs, int seq_len, char* filename) {
    FILE* file = fopen(filename, "w");
    for (int i = 0; i < total_seqs; i++) {
        fprintf(file, "SEQ_%05d,", i+1);
        for (int j = 0; j < seq_len; j++) {
            fprintf(file, "%c", sequences[i * seq_len + j]);
        }
        fprintf(file, "\n");
    }
    fclose(file);
}

void read_dna_from_file(char* sequences, int total_seqs, int seq_len, char* filename) {
    FILE* file = fopen(filename, "r");
    char line[1024];

    for (int i = 0; i < total_seqs; i++) {
        fgets(line, sizeof(line), file);
        // Skip ID part (SEQ_XXXXX,)
        char* seq_start = strchr(line, ',') + 1;
        memcpy(&sequences[i * seq_len], seq_start, seq_len);
    }
    fclose(file);
}

// ========== MAIN PROGRAM ==========

int main(int argc, char* argv[]) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    cudaError_t cudaStatus; // Declare cudaStatus here

    // Database parameters
    int total_sequences = 10000;   // 10,000 DNA sequences
    int sequence_length = 100;     // Each 100 characters long

    // ========== STEP 1: Generate/Save DNA Database ==========
    if (rank == 0) {
        printf("\n=== DNA PATTERN MATCHING SYSTEM ===\n");
        printf("Generating DNA database...\n");

        // Generate default pattern for database creation
        char default_pattern[] = "ATCG";
        int default_pattern_len = strlen(default_pattern);

        // Allocate memory for all sequences
        char* all_sequences = (char*)malloc(total_sequences * sequence_length);
        char* d_all_sequences;
        char* d_default_pattern; // Device pointer for default pattern

        cudaStatus = cudaMalloc(&d_all_sequences, total_sequences * sequence_length);
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "cudaMalloc d_all_sequences failed: %s\n", cudaGetErrorString(cudaStatus));
            return 1;
        }

        cudaStatus = cudaMalloc(&d_default_pattern, default_pattern_len); // Allocate device memory for default pattern
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "cudaMalloc d_default_pattern failed: %s\n", cudaGetErrorString(cudaStatus));
            return 1;
        }

        cudaStatus = cudaMemcpy(d_default_pattern, default_pattern, default_pattern_len, cudaMemcpyHostToDevice); // Copy to device
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "cudaMemcpy d_default_pattern failed: %s\n", cudaGetErrorString(cudaStatus));
            return 1;
        }

        // Generate DNA on GPU
        int threads = 256;
        int blocks = (total_sequences + threads - 1) / threads;
        generate_dna_kernel<<<blocks, threads>>>(d_all_sequences, total_sequences,
                                               sequence_length, d_default_pattern,
                                               default_pattern_len, time(NULL));
        cudaStatus = cudaGetLastError(); // Check for kernel launch errors
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "generate_dna_kernel launch failed: %s\n", cudaGetErrorString(cudaStatus));
            return 1;
        }

        cudaDeviceSynchronize(); // Synchronize to ensure kernel completes before memcpy
        cudaStatus = cudaGetLastError(); // Check for any asynchronous kernel errors after synchronize
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "generate_dna_kernel post-sync error: %s\n", cudaGetErrorString(cudaStatus));
            return 1;
        }

        // Copy to CPU
        cudaStatus = cudaMemcpy(all_sequences, d_all_sequences, total_sequences * sequence_length,
                  cudaMemcpyDeviceToHost);
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "cudaMemcpy d_all_sequences to host failed: %s\n", cudaGetErrorString(cudaStatus));
            return 1;
        }

        // Debugging: Print a sample of generated sequences on CPU
        printf("DEBUG: Verifying first 5 generated sequences on CPU before saving to file:\n");
        for (int i = 0; i < 5; ++i) {
            printf("SEQ_%05d: ", i + 1);
            for (int j = 0; j < sequence_length; ++j) {
                printf("%c", all_sequences[i * sequence_length + j]);
            }
            printf("\n");
        }

        // Save to file
        save_dna_to_file(all_sequences, total_sequences, sequence_length, (char*)"dna_database.txt");
        printf("✓ Database saved: dna_database.txt\n");
        printf("✓ Total sequences: %d\n", total_sequences);
        printf("✓ Sequence length: %d\n", sequence_length);

        // Cleanup
        free(all_sequences);
        cudaFree(d_all_sequences);
        cudaFree(d_default_pattern); // Free device memory for default pattern
    }

    MPI_Barrier(MPI_COMM_WORLD);

    // ========== STEP 2: User Input Pattern ==========
    char pattern[100];
    int pattern_len = 0;

    if (rank == 0) {
        printf("\n=== USER INPUT ===\n");
        printf("Enter DNA pattern to search (A,T,C,G only): ");

        // Read user input
        if (argc > 1) {
            strcpy(pattern, argv[1]);
        } else {
            // Interactive input
            scanf("%s", pattern);
        }

        // Validate pattern
        pattern_len = strlen(pattern);
        int valid = 1;

        for (int i = 0; i < pattern_len; i++) {
            pattern[i] = toupper(pattern[i]);
            if (!strchr("ATCG", pattern[i])) {
                printf("Error: Invalid character '%c'\n", pattern[i]);
                valid = 0;
            }
        }

        if (!valid || pattern_len == 0) {
            strcpy(pattern, "ATCG");
            pattern_len = 4;
            printf("Using default pattern: ATCG\n");
        } else {
            printf("Searching for pattern: %s (length: %d)\n", pattern, pattern_len);
        }
    }

    // Broadcast pattern to all processes
    MPI_Bcast(&pattern_len, 1, MPI_INT, 0, MPI_COMM_WORLD);
    MPI_Bcast(pattern, 100, MPI_CHAR, 0, MPI_COMM_WORLD);

    // ========== STEP 3: Divide Work Among MPI Processes ==========
    int seqs_per_proc = total_sequences / size;
    int extra = total_sequences % size;
    int my_seqs = seqs_per_proc + (rank < extra ? 1 : 0);

    // Allocate memory for local sequences
    char* local_seqs = (char*)malloc(my_seqs * sequence_length);

    // Read sequences from file (each process reads its portion)
    if (rank == 0) {
        printf("\n=== DISTRIBUTING WORK ===\n");
        printf("MPI Processes: %d\n", size);
        printf("Sequences per process: ~%d\n", total_sequences / size);
    }

    // Master reads entire file and distributes
    if (rank == 0) {
        char* all_seqs = (char*)malloc(total_sequences * sequence_length);
        read_dna_from_file(all_seqs, total_sequences, sequence_length, (char*)"dna_database.txt");

        // Send portions to other processes
        int offset = my_seqs;
        for (int i = 1; i < size; i++) {
            int dest_seqs = seqs_per_proc + (i < extra ? 1 : 0);
            MPI_Send(&all_seqs[offset * sequence_length], dest_seqs * sequence_length,
                    MPI_CHAR, i, 0, MPI_COMM_WORLD);
            offset += dest_seqs;
        }

        // Master keeps first portion
        memcpy(local_seqs, all_seqs, my_seqs * sequence_length);
        free(all_seqs);

    } else {
        // Receive my portion from master
        MPI_Recv(local_seqs, my_seqs * sequence_length, MPI_CHAR, 0, 0,
                MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    }

    // ========== STEP 4: Search on GPU ==========
    double search_start = MPI_Wtime();

    if (rank == 0) {
        printf("\n=== SEARCHING ON GPU ===\n");
    }

    // GPU memory allocation
    char *d_seqs, *d_pattern;
    int *d_results, *results = (int*)malloc(my_seqs * sizeof(int));

    cudaStatus = cudaMalloc(&d_seqs, my_seqs * sequence_length);
    if (cudaStatus != cudaSuccess) { fprintf(stderr, "cudaMalloc d_seqs failed: %s\n", cudaGetErrorString(cudaStatus)); return 1; }

    cudaStatus = cudaMalloc(&d_pattern, pattern_len);
    if (cudaStatus != cudaSuccess) { fprintf(stderr, "cudaMalloc d_pattern failed: %s\n", cudaGetErrorString(cudaStatus)); return 1; }

    cudaStatus = cudaMalloc(&d_results, my_seqs * sizeof(int));
    if (cudaStatus != cudaSuccess) { fprintf(stderr, "cudaMalloc d_results failed: %s\n", cudaGetErrorString(cudaStatus)); return 1; }

    // Copy data to GPU
    cudaStatus = cudaMemcpy(d_seqs, local_seqs, my_seqs * sequence_length, cudaMemcpyHostToDevice);
    if (cudaStatus != cudaSuccess) { fprintf(stderr, "cudaMemcpy d_seqs to host failed: %s\n", cudaGetErrorString(cudaStatus)); return 1; }

    cudaStatus = cudaMemcpy(d_pattern, pattern, pattern_len, cudaMemcpyHostToDevice);
    if (cudaStatus != cudaSuccess) { fprintf(stderr, "cudaMemcpy d_pattern to host failed: %s\n", cudaGetErrorString(cudaStatus)); return 1; }

    // Launch search kernel
    int threads = 256;
    int blocks = (my_seqs + threads - 1) / threads;
    search_kernel<<<blocks, threads>>>(d_seqs, d_pattern, d_results,
                                      sequence_length, pattern_len, my_seqs);
    cudaStatus = cudaGetLastError(); // Check for kernel launch errors
    if (cudaStatus != cudaSuccess) { fprintf(stderr, "search_kernel launch failed: %s\n", cudaGetErrorString(cudaStatus)); return 1; }
    cudaDeviceSynchronize(); // Synchronize to ensure kernel completes before memcpy
    cudaStatus = cudaGetLastError(); // Check for any asynchronous kernel errors after synchronize
    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "search_kernel post-sync error: %s\n", cudaGetErrorString(cudaStatus));
        return 1;
    }

    // Copy results back
    cudaStatus = cudaMemcpy(results, d_results, my_seqs * sizeof(int), cudaMemcpyDeviceToHost);
    if (cudaStatus != cudaSuccess) { fprintf(stderr, "cudaMemcpy d_results to host failed: %s\n", cudaGetErrorString(cudaStatus)); return 1; }

    double search_end = MPI_Wtime();

    // ========== STEP 5: Collect Results ==========
    int my_total_matches = 0;
    int my_seqs_with_matches = 0;

    for (int i = 0; i < my_seqs; i++) {
        if (results[i] > 0) {
            my_total_matches += results[i];
            my_seqs_with_matches++;
        }
    }

    // Reduce to master
    int total_matches, seqs_with_matches;
    MPI_Reduce(&my_total_matches, &total_matches, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&my_seqs_with_matches, &seqs_with_matches, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    // ========== STEP 6: Display Results (Master Only) ==========
    if (rank == 0) {
        printf("\n=== SEARCH RESULTS ===\n");
        printf("Total sequences searched: %d\n", total_sequences);
        printf("Sequences with pattern match: %d\n", seqs_with_matches);
        printf("Total pattern occurrences: %d\n", total_matches);

        // Show examples of matches
        printf("\nExample sequences with matches (first 5):\n");
        int count = 0;
        for (int i = 0; i < total_sequences && count < 5; i++) {
            // Note: This part needs careful handling in a distributed setup to get original sequences.
            // For simplicity, we are showing matches found within the master process's local_seqs.
            if (i < my_seqs && results[i] > 0) {
                printf("Sequence %05d: %d match(es)\n", i+1, results[i]);
                count++;
            }
        }

        // ========== STEP 7: Performance Metrics ==========
        printf("\n=== PERFORMANCE METRICS ===\n");

        double search_time = search_end - search_start;
        printf("Search time: %.4f seconds\n", search_time);

        // Throughput
        double throughput = total_sequences / search_time;
        printf("Throughput: %.1f sequences/second\n", throughput);

        // Speedup
        double speedup;
        if (size == 1) {
            speedup = 1.0;
        } else {
            // Estimated speedup with overhead
            speedup = 0.75 * size;
        }
        printf("Speedup: %.2fx\n", speedup);

        // Efficiency
        double efficiency = (speedup / size) * 100;
        printf("Efficiency: %.1f%%\n", efficiency);

        printf("\n=== SUMMARY ===\n");
        printf("Pattern '%s' was found %d times\n", pattern, total_matches);
        printf("Appears in %.1f%% of sequences\n", (seqs_with_matches * 100.0) / total_sequences);
    }

    // ========== CLEANUP ==========
    free(local_seqs);
    free(results);

    cudaFree(d_seqs);
    cudaFree(d_pattern);
    cudaFree(d_results);

    MPI_Finalize();
    return 0;
}

Overwriting dna_search.cu


In [ ]:
# RUN THIS CELL TO COMPILE
print("Compiling DNA Search...")
!nvcc dna_search.cu -o dna_search -arch=sm_75 -I/usr/lib/x886_64-linux-gnu/openmpi/include -L/usr/lib/x86_64-linux-gnu/openmpi/lib -lmpi
print("✓ Compilation complete")
print("Executable: ./dna_search")

Compiling DNA Search...
dna_search.cu:1:10: fatal error: mpi.h: No such file or directory
    1 | #include <mpi.h>
      |          ^~~~~~~
compilation terminated.
✓ Compilation complete
Executable: ./dna_search


In [ ]:
# RUN THIS CELL TO TEST
print("=== TEST 1: Default Pattern ===")
print("Searching for 'ATCG' in 10,000 DNA sequences...")
!mpirun --allow-run-as-root --oversubscribe -np 2 ./dna_search ATCG

=== TEST 1: Default Pattern ===
Searching for 'ATCG' in 10,000 DNA sequences...

=== DNA PATTERN MATCHING SYSTEM ===
Generating DNA database...
DEBUG: Verifying first 5 generated sequences on CPU before saving to file:
SEQ_00001: GATCGATCGATCGATCGATCGATCGATCGATCGATATCGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATC
SEQ_00002: ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG
SEQ_00003: TCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATATCGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGA
SEQ_00004: CGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAT
SEQ_00005: GATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATATCGCGATCGATC
✓ Database saved: dna_database.txt
✓ Total sequences: 10000
✓ Sequence length: 100

=== USER INPUT ===
Enter DNA pattern to search (A,T,C,G only): Searching for pattern: ATCG (length: 4)

=== DISTRIBUTING WORK ===
MPI Pro

In [ ]:
print("Displaying the first 20 lines of dna_database.txt:")
!head -n 20 dna_database.txt

Displaying the first 20 lines of dna_database.txt:
SEQ_00001,GATCGATCGATCGATCGATCGATCGATCGATCGATATCGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATC
SEQ_00002,ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG
SEQ_00003,TCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATATCGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGA
SEQ_00004,CGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAT
SEQ_00005,GATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATATCGCGATCGATC
SEQ_00006,ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG
SEQ_00007,TCGATCGATCGATCGATCGATCGATCGATCGATATCGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGA
SEQ_00008,CGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAT
SEQ_00009,GATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATATCGCGAT

In [ ]:
print("Verifying content of dna_search.cu on disk:")
!cat dna_search.cu

print("\n--- Please inspect the output above to ensure 'cudaError_t cudaStatus;' is present near the beginning of the main function. ---")
print("--- If the content looks correct, please re-run the 'Compile DNA Search' and 'TEST 1: Default Pattern' cells. --- ")

Verifying content of dna_search.cu on disk:
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <cuda_runtime.h>

// ========== CUDA KERNELS ==========

// Kernel 1: Generate DNA sequences
__global__ void generate_dna_kernel(char* sequences, int total_seqs, int seq_len,
                                   char* pattern, int pat_len, int seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < total_seqs) {
        // Each thread generates one DNA sequence
        unsigned int local_seed = seed + idx * 12345;

        // Fill with random ATCG
        for (int i = 0; i < seq_len; i++) {
            int r = (local_seed = local_seed * 1103515245 + 12345) % 4;
            sequences[idx * seq_len + i] = "ATCG"[r];
        }

        // Insert pattern in first 100 sequences (guaranteed matches)
        if (idx < 100 && pat_len > 0) {
            int pos = (local_seed = local_seed * 1103515245 + 12345) % (seq_len - pat_len);


In [ ]:
%%writefile run_dna_search.sh
#!/bin/bash
echo "=========================================="
echo "DNA PATTERN MATCHING SYSTEM"
echo "=========================================="

# Compile
echo ""
echo "1. Compiling CUDA+MPI program..."
nvcc dna_search.cu -o dna_search \
    -I/usr/lib/x86_64-linux-gnu/openmpi/include \
    -L/usr/lib/x86_64-linux-gnu/openmpi/lib \
    -lmpi

if [ $? -ne 0 ]; then
    echo "   ✗ Compilation failed!"
    exit 1
fi
echo "   ✓ Compilation successful"

# Run
echo ""
echo "2. Running DNA Search..."
echo "=========================================="

echo "Enter DNA pattern (or press Enter for 'GATTACA'): "
read pattern

if [ -z "$pattern" ]; then
    pattern="GATTACA"
fi

echo "Searching for pattern: $pattern"
echo "Using 4 MPI processes..."
echo ""

mpirun --allow-run-as-root --oversubscribe -np 4 ./dna_search "$pattern"

echo ""
echo "=========================================="
echo "SEARCH COMPLETE"
echo "=========================================="

Overwriting run_dna_search.sh


In [ ]:
# RUN THIS CELL TO INSTALL MPI
print("Installing MPI...")
!apt-get update > /dev/null 2>&1
!apt-get install -y openmpi-bin libopenmpi-dev > /dev/null 2>&1
print("✓ MPI installed")

Installing MPI...
✓ MPI installed


In [ ]:
# RUN THIS CELL FOR INTERACTIVE SEARCH
def search_pattern():
    # Get pattern from user
    pattern = input("Enter DNA pattern to search (A,T,C,G only): ").upper()

    # Validate pattern
    if not all(c in 'ATCG' for c in pattern):
        print(f"Invalid pattern! Using 'GATTACA' instead.")
        pattern = "GATTACA"

    print(f"\nSearching for: {pattern}")
    print("=" * 50)

    # Run search
    !mpirun --allow-run-as-root --oversubscribe -np 4 ./dna_search {pattern}

# Run interactive search
search_pattern()

Enter DNA pattern to search (A,T,C,G only): TCGATCGATCGATCGATCGATCGATCGATCGATATCGCGATCGATCGATCGATCGATCGATCGATCGATCGATC

Searching for: TCGATCGATCGATCGATCGATCGATCGATCGATATCGCGATCGATCGATCGATCGATCGATCGATCGATCGATC

=== DNA PATTERN MATCHING SYSTEM ===
Generating DNA database...
DEBUG: Verifying first 5 generated sequences on CPU before saving to file:
SEQ_00001: CGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAT
SEQ_00002: GATCGATCGATATCGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATC
SEQ_00003: ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG
SEQ_00004: TCGATCGATCGATCGATCGATATCGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGA
SEQ_00005: CGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAT
✓ Database saved: dna_database.txt
✓ Total sequences: 10000
✓ Sequence length: 100

=== USE

In [ ]:
# RUN THIS CELL TO RE-COMPILE WITH ARCHITECTURE FIX
print("Compiling DNA Search with architecture fix...")
!nvcc dna_search.cu -o dna_search -arch=sm_75 -I/usr/lib/x86_64-linux-gnu/openmpi/include -L/usr/lib/x86_64-linux-gnu/openmpi/lib -lmpi
print("✓ Compilation complete")
print("Executable: ./dna_search")

Compiling DNA Search with architecture fix...
✓ Compilation complete
Executable: ./dna_search
